In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import time
#import xgboost as xgb
from IPython.display import display
train = pd.read_csv('../input/train.csv')
#display(train.head())
print('Number of NaN values')
#check if there are any missing values
#display(pd.isnull(train).sum())
#Parsing DT string
train['pickup_datetime'] = train['pickup_datetime'].astype(str)
dtparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
train['pickup_datetime'] = train['pickup_datetime'].apply(dtparse)
train['DOW'] = train['pickup_datetime'].dt.dayofweek
train['Hour'] = train['pickup_datetime'].apply(lambda x:x.hour)
display(train['pickup_datetime'].describe())
display(train.head())

from math import radians, sin, cos, sqrt, asin
def haversine(columns):
  lat1, lon1, lat2, lon2 = columns
  R = 6372.8 # Earth radius in kilometers

  dLat = radians(lat2 - lat1)
  dLon = radians(lon2 - lon1)
  lat1 = radians(lat1)
  lat2 = radians(lat2)
  a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
  c = 2*asin(sqrt(a))

  return R * c
cols = ['pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude']
distances = train[cols].apply(
    lambda x: haversine(x),axis = 1
)
train['haversine_distances'] = distances.copy()
train.head()

lowq,highq = 1,99

#Trimming based on outlier distances and durations
trip = train['trip_duration']
ltrip,rtrip= np.percentile(trip,[lowq,highq])#getting 1 and 99 percentiles
print(ltrip,rtrip)
train_trimmed = train[trip.between(ltrip,rtrip)]

haversine=train['haversine_distances']
ltrip,rtrip= np.percentile(haversine,[lowq,highq])
print(ltrip,rtrip)

train_trimmed = train_trimmed[haversine.between(ltrip,rtrip)]

features = train_trimmed[['DOW','Hour','passenger_count','pickup_latitude','pickup_longitude']]
target = train_trimmed[['trip_duration']]

from sklearn.cluster import KMeans
import numpy as np
import pickle

try:
    kmeans = pickle.load(open("source_kmeans.pickle", "rb"))
except:
    kmeans = KMeans(n_clusters=20, random_state=0).fit(train_trimmed[['pickup_longitude','pickup_latitude']])
    pickle.dump(kmeans, open('source_kmeans.pickle', 'wb'))

try:
    destkmeans = pickle.load(open("dest_kmeans.pickle", "rb"))
except:
    destkmeans = KMeans(n_clusters=20, random_state=0).fit(train_trimmed[['dropoff_longitude','dropoff_latitude']])
    pickle.dump(destkmeans, open('dest_kmeans.pickle', 'wb'))


train_trimmed['cluster'] = kmeans.predict(train_trimmed[['pickup_longitude','pickup_latitude']])
train_trimmed['dest_cluster'] = destkmeans.predict(train_trimmed[['dropoff_longitude','dropoff_latitude']])


import xgboost


from sklearn import linear_model
features = train_trimmed[['DOW','Hour','passenger_count','cluster','dest_cluster','dropoff_latitude','dropoff_longitude','pickup_longitude','pickup_latitude']]
target = train_trimmed[['trip_duration']]
reg = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

reg.fit(features,target)


 
tdf = pd.read_csv('../input/test.csv')
tdf['cluster'] = kmeans.predict(tdf[['pickup_longitude','pickup_latitude']])
tdf['dest_cluster'] = destkmeans.predict(tdf[['dropoff_longitude','dropoff_latitude']])
tdf.pickup_datetime=pd.to_datetime(tdf.pickup_datetime)
#tdf.dropoff_datetime=pd.to_datetime(tdf.dropoff_datetime)
tdf['Hour'] = tdf.pickup_datetime.dt.hour

tdf['DOW'] = tdf.pickup_datetime.dt.dayofweek
tfeatures = tdf[['DOW','Hour','passenger_count','cluster','dest_cluster','dropoff_latitude','dropoff_longitude','pickup_longitude','pickup_latitude']]
pred = reg.predict(tfeatures)



tdf['trip_duration']=pred.astype(int)
out = tdf[['id','trip_duration']]
out['trip_duration'].isnull().values.any()
out.to_csv('pred_linear_2_clusters.csv',index=False)



print('completed')